##Importação das bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box
import plotly.graph_objects as go
from shapely.geometry import Point, Polygon
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

##Pandas

In [ ]:
data = pd.read_csv('dataset-limpo1.csv')
data.head()

,id,bairro,created_at,descricao,endereco,id.1,ip_address_origin,latitude,longitude,registrou_bo,...,Estepe,MP4 ou Ipod,Móveis,Notebook,Outros,Relógio,Som,Tablet,Tv,time
0,49787,Butantã,34:01.0,"5 bandidos, armas, entraram em casa e vasculha...","Avenida Trona Constanzo, 177, Butantã",49787,NaN,-23.580973,-46.719192,True,...,NaN,True,NaN,True,NaN,NaN,True,True,True,30:00.0
1,10679,Itaquera,26:31.8,"Meu caso foi para o Pequenas Causas, ganhamos ...","Avenida José Pinheiro Borges, 450-876 - Itaque...",10679,187.11.1.51,-23.540105,-46.462872,True,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,00:00.0
2,10682,Itaquera,42:07.9,"Meu caso foi para o Pequenas Causas, ganhamos ...",Rua Corínthians - Itaquera - Shopping Metrô It...,10682,189.19.160.82,-23.541765,-46.469562,True,...,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,00:00.0
3,6166,Morumbi,56:52.7,"Estava parada no semáforo, o bandido veio e es...","Rua Pasquale Gallupi, 347 - Morumbi, São Paulo...",6166,199.67.140.46,-23.611693,-46.725935,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:00.0
4,6521,Alto de Pinheiros,41:21.6,Arrombaram o porta malas do meu carro e levara...,"Avenida Professor Fonseca Rodrigues, 1655 - Al...",6521,187.92.158.154,-23.544564,-46.721567,True,...,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,15:00.0


In [ ]:
data.isnull().sum()

id                        0
bairro                 1703
created_at                0
descricao                 1
endereco                  0
id.1                      0
ip_address_origin      1194
latitude                  0
longitude                 0
registrou_bo            524
sexo                      0
tipo_assalto_id           0
titulo                    0
uri                       0
user_id               11733
valor_prejuizo            0
Bicicleta             12643
Bolsa ou Mochila       9268
Carteira               8656
Cartão de Crédito     10123
Celular                4522
Computador            12787
DVD                   12805
Dinheiro              11169
Documentos             9239
Equipamento de Som    12660
Estepe                12575
MP4 ou Ipod           12347
Móveis                12854
Notebook              12135
Outros                 7656
Relógio               11549
Som                   12793
Tablet                12456
Tv                    12721
time                

In [ ]:
df = data.dropna(subset=['bairro'])
df.isnull().sum()

id                        0
bairro                    0
created_at                0
descricao                 1
endereco                  0
id.1                      0
ip_address_origin      1171
latitude                  0
longitude                 0
registrou_bo            524
sexo                      0
tipo_assalto_id           0
titulo                    0
uri                       0
user_id               10190
valor_prejuizo            0
Bicicleta             10992
Bolsa ou Mochila       8021
Carteira               7457
Cartão de Crédito      8760
Celular                3906
Computador            11099
DVD                   11111
Dinheiro               9957
Documentos             7990
Equipamento de Som    10970
Estepe                10902
MP4 ou Ipod           10695
Móveis                11155
Notebook              10516
Outros                 6643
Relógio               10000
Som                   11101
Tablet                10804
Tv                    11042
time                

##Plotly

In [ ]:
fig = px.density_mapbox(df, lat='latitude', lon='longitude', radius=5, zoom=7,
                        color_continuous_scale=px.colors.sequential.YlOrRd,
                        mapbox_style="stamen-terrain")
fig.update_layout(
        title = 'Mapa densidade de crimes',
)
fig.show()

##GeoPandas

In [ ]:
# Converter o DataFrame em um Geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))


# Definir as regiões que você deseja usar
regions = {
    'norte': {'latitude_min': -23.0, 'latitude_max': -22.0, 'longitude_min': -48.0, 'longitude_max': -45.0},
    'leste': {'latitude_min': -24.0, 'latitude_max': -23.0, 'longitude_min': -48.0, 'longitude_max': -45.0},
    'sul': {'latitude_min': -25.0, 'latitude_max': -24.0, 'longitude_min': -48.0, 'longitude_max': -45.0},
    'oeste': {'latitude_min': -23.0, 'latitude_max': -22.0, 'longitude_min': -53.0, 'longitude_max': -48.0}
}

# Função para atribuir a região com base nas coordenadas
def assign_region(row):
    for region, bounds in regions.items():
        if bounds['latitude_min'] <= row['latitude'] <= bounds['latitude_max'] and bounds['longitude_min'] <= row['longitude'] <= bounds['longitude_max']:
            return region
    return 'região desconhecida'

# Aplicar a função para criar a coluna de regiões
gdf['região'] = gdf.apply(assign_region, axis=1)


In [ ]:
region_counts = gdf['região'].value_counts()
print(region_counts)

leste                  11170
região desconhecida       10
sul                        9
norte                      4
oeste                      3
Name: região, dtype: int64


###No conjunto de dados fornecido, a maioria dos assaltos ocorreu na região 2 (leste do estado de São Paulo), com um total de 11.170 assaltos registrados. Em seguida, temos a região 3 com 9 assaltos, a região 1 com 4 assaltos e a região 4 com 3 assaltos. Além disso, existem 10 assaltos que não puderam ser atribuídos a nenhuma região específica e foram classificados como "região desconhecida

##Criação subdivisão da região leste do estado de são paulo(região mais perigosa)

In [ ]:
regions_subdivided = {
    'leste - norte': {'latitude_min': -23.5, 'latitude_max': -23.0, 'longitude_min': -47.5, 'longitude_max': -47.0},
    'leste - leste': {'latitude_min': -23.5, 'latitude_max': -23.0, 'longitude_min': -47.0, 'longitude_max': -46.5},
    'leste - sul': {'latitude_min': -24.0, 'latitude_max': -23.5, 'longitude_min': -47.5, 'longitude_max': -47.0},
    'leste - oeste': {'latitude_min': -24.0, 'latitude_max': -23.5, 'longitude_min': -47.0, 'longitude_max': -46.5}
}


In [ ]:
# Função para atribuir a sub-região com base nas coordenadas
def assign_subregiao(row):
    for subregiao, bounds in regions_subdivided.items():
        if bounds['latitude_min'] <= row['latitude'] <= bounds['latitude_max'] and bounds['longitude_min'] <= row['longitude'] <= bounds['longitude_max']:
            return subregiao
    return 'sub-região desconhecida'

# Aplicar a função para criar a coluna de sub-regiões
gdf['subregião_2'] = gdf.apply(assign_subregiao, axis=1)

# Contar as ocorrências de cada sub-região
subregiao_counts = gdf['subregião_2'].value_counts()
print(subregiao_counts)


leste - oeste              9428
sub-região desconhecida     986
leste - leste               778
leste - norte                 3
leste - sul                   1
Name: subregião_2, dtype: int64


In [ ]:
# Filtrar o GeoDataFrame para incluir apenas os registros da sub-região "leste - oeste"
gdf_leste_oeste = gdf[gdf['subregião_2'] == 'leste - oeste']

# Criar o gráfico de dispersão geográfico
fig = px.scatter_mapbox(
    data_frame=gdf_leste_oeste,
    lat='latitude',
    lon='longitude',
    color='região',
    mapbox_style='open-street-map',
    zoom=10
)

# Exibir o gráfico
fig.show()

###A região mais perigosa é o oeste do estado de são paulo, no centro da cidade de são paulo. Sendo um polo de assaltos.

In [ ]:
gdf.head()

,id,bairro,created_at,descricao,endereco,id.1,ip_address_origin,latitude,longitude,registrou_bo,...,Notebook,Outros,Relógio,Som,Tablet,Tv,time,geometry,região,subregião_2
0,49787,Butantã,34:01.0,"5 bandidos, armas, entraram em casa e vasculha...","Avenida Trona Constanzo, 177, Butantã",49787,NaN,-23.580973,-46.719192,True,...,True,NaN,NaN,True,True,True,30:00.0,POINT (-46.71919 -23.58097),leste,leste - oeste
1,10679,Itaquera,26:31.8,"Meu caso foi para o Pequenas Causas, ganhamos ...","Avenida José Pinheiro Borges, 450-876 - Itaque...",10679,187.11.1.51,-23.540105,-46.462872,True,...,NaN,True,NaN,NaN,NaN,NaN,00:00.0,POINT (-46.46287 -23.54011),leste,sub-região desconhecida
2,10682,Itaquera,42:07.9,"Meu caso foi para o Pequenas Causas, ganhamos ...",Rua Corínthians - Itaquera - Shopping Metrô It...,10682,189.19.160.82,-23.541765,-46.469562,True,...,NaN,True,NaN,NaN,NaN,NaN,00:00.0,POINT (-46.46956 -23.54177),leste,sub-região desconhecida
3,6166,Morumbi,56:52.7,"Estava parada no semáforo, o bandido veio e es...","Rua Pasquale Gallupi, 347 - Morumbi, São Paulo...",6166,199.67.140.46,-23.611693,-46.725935,True,...,NaN,NaN,NaN,NaN,NaN,NaN,20:00.0,POINT (-46.72593 -23.61169),leste,leste - oeste
4,6521,Alto de Pinheiros,41:21.6,Arrombaram o porta malas do meu carro e levara...,"Avenida Professor Fonseca Rodrigues, 1655 - Al...",6521,187.92.158.154,-23.544564,-46.721567,True,...,True,True,NaN,NaN,NaN,NaN,15:00.0,POINT (-46.72157 -23.54456),leste,leste - oeste


In [ ]:
# Coordenadas do centro de São Paulo
center_lat = -23.5505
center_lon = -46.6333

# Criar o hexágono ao redor do centro de São Paulo
hexagon_radius = 0.02  # Ajuste o raio do hexágono conforme necessário

# Criar as coordenadas do hexágono
hexagon_coords = [
    (center_lon + hexagon_radius, center_lat),
    (center_lon + hexagon_radius / 2, center_lat + hexagon_radius * 3 ** 0.5 / 2),
    (center_lon - hexagon_radius / 2, center_lat + hexagon_radius * 3 ** 0.5 / 2),
    (center_lon - hexagon_radius, center_lat),
    (center_lon - hexagon_radius / 2, center_lat - hexagon_radius * 3 ** 0.5 / 2),
    (center_lon + hexagon_radius / 2, center_lat - hexagon_radius * 3 ** 0.5 / 2),
    (center_lon + hexagon_radius, center_lat)
]

# Criar o hexágono como um Polygon
hexagon = Polygon(hexagon_coords)

# Filtrar os pontos dentro do hexágono
points_within_hexagon = gdf[gdf.geometry.within(hexagon)]

# Criar o gráfico de dispersão
fig = go.Figure()

# Adicionar os pontos dentro do hexágono
fig.add_trace(go.Scattermapbox(
    lat=points_within_hexagon['latitude'],
    lon=points_within_hexagon['longitude'],
    mode='markers',
    marker=dict(size=6, color='blue')
))

# Adicionar o hexágono ao redor do centro de São Paulo
fig.add_trace(go.Scattermapbox(
    lat=[coord[1] for coord in hexagon_coords + [hexagon_coords[0]]],
    lon=[coord[0] for coord in hexagon_coords + [hexagon_coords[0]]],
    mode='lines',
    line=dict(color='red', width=2),
    fill='toself',
    fillcolor='rgba(255, 0, 0, 0.2)',
    showlegend=False
))

# Configurar layout do mapa
fig.update_layout(
    mapbox=dict(
        style="stamen-terrain",  # Estilo do mapa (pode ser ajustado conforme preferência)
        center=dict(lat=center_lat, lon=center_lon),  # Centro do mapa
        zoom=14  # Zoom do mapa
    ),
    showlegend=False
)

# Exibir o gráfico
fig.show()

In [ ]:
df = gdf

##Criação de ML para classificar uma região para cada ponto

---



In [ ]:
# Dividir os dados em treinamento e teste
X = df[['latitude', 'longitude']]
y = df['região']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar o modelo KNN e ajustá-lo aos dados de treinamento
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Fazer as previsões da região para os pontos de teste
predictions = knn.predict(X_test)

# Verificar a acurácia do modelo
accuracy = knn.score(X_test, y_test)

print(f'Acurácia: {accuracy * 100:.2f}%')

Acurácia: 99.82%
